## Item 27: Use Comprehensions Instead of map and filter

In [2]:
a = [1, 2, 3]

hard_to_read = map(lambda x: x**2, filter(lambda x: x % 2 == 0, a))
better = [x**2 for x in a if x % 2 == 0]

print(list(hard_to_read))
print(better)

[4]
[4]


In [3]:
squares_dict = {x: x**2 for x in a}
cubes_set = {x**3 for x in a}

print(squares_dict)
print(cubes_set)

{1: 1, 2: 4, 3: 9}
{8, 1, 27}


## Item 28: Avoid More Than Two Control Subexpressions in Comprehensions

In [4]:
matrix = [[1, 2], [3, 4], [5, 6]]
flat = [x for row in matrix for x in row]

print(flat)

[1, 2, 3, 4, 5, 6]


In [5]:
squared = [[x**2 for x in row] for row in matrix]

print(squared)

[[1, 4], [9, 16], [25, 36]]


In [6]:
"""
This is hard to read.

The rule of thumb is to avoid using more than 
two control subexpressions in a comprehension. 
This could be two conditions, two loops, or 
one condition and one loop.
"""
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
filtered = [[x for x in row if x % 3 == 0]
            for row in matrix if sum(row) >= 10]
print(filtered)

[[6], [9]]


## Item 29: Avoid Repeated Work in Comprehensions by Using Assignment Expressions

You can remove repetition using the Walrus operator.

```python
bad = found = {name: get_batches(stock.get(name, 0), 8)
         for name in order
         if get_batches(stock.get(name, 0), 8)}

good = found = {name: batches for name in order
         if (batches := get_batches(stock.get(name, 0), 8))}
```

You cannot reference the variable it defines in other parts of the comprehension.

```python
result = {name: (tenth := count // 10)
          for name, count in stock.items() if tenth > 0}

>>>
Traceback ...
NameError: name 'tenth' is not defined
```

Fix:
```python
result = {name: tenth for name, count in stock.items()
          if (tenth := count // 10) > 0}
print(result)

>>>
{'nails': 12, 'screws': 3, 'washers': 2}
```

## Item 30: Consider Generators Instead of Returning Lists

In [5]:
"""
This is a bit noisy to read due to the list.
"""
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter == ' ':
            result.append(index + 1)
    return result


address = 'Four score and seven years ago...'
result = index_words(address)
print(result[:10])

[0, 5, 11, 15, 21, 27]


In [6]:
"""
This is easier to read and 
it doesn't consume huge memory for large input.
"""
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
             yield index + 1
                

it = index_words_iter(address)
print(next(it))
print(next(it))
result = list(index_words_iter(address))
print(result[:10])      

0
5
[0, 5, 11, 15, 21, 27]


## Item 31: Be Defensive When Iterating Over Arguments

In [9]:
"""
The iterator protocol is how Python for loops and related expressions
traverse the contents of a container type.
"""
a = [1]

it = a.__iter__()
print(it.__next__())
try:
    it.__next__()
except StopIteration as e:
    print(f'{e!r}')

1
StopIteration()


In [15]:
"""
The protocol states that when an iterator is passed to 
the iter built-in function, iter returns the iterator itself. 
In contrast, when a container type is passed to iter, 
a new iterator object is returned each time.
"""

it1 = iter(a)
it2 = iter(it1)
it3 = iter(it2)
it4 = iter(it3)

print(it1)
print(it2)
print(it3)
print(it1 is it3) 

True


In [16]:
from collections.abc import Iterator

def isiterator1(v):
    return iter(v) is v

def isiterator2(v):
    return isinstance(v, Iterator)

print(isiterator1(a))
print(isiterator1(it))

print(isiterator2(a))
print(isiterator2(it))

False
True
False
True


In [17]:
def multi_iter(n):
    return sum(n) + sum(n)

print(multi_iter([1,2,3]))
print(multi_iter(iter([1,2,3]))) # exhausted iterator problem

12
6


In [30]:
def multi_iter_defensive(n):
    if isinstance(n, Iterator):
        raise TypeError('Must supply a container')
    return sum(n) + sum(n)


print(multi_iter_defensive([1,2,3]))

try:
    multi_iter_defensive(iter([1,2,3]))
except TypeError as e:
    print(f'{e!r}')

12
TypeError('Must supply a container')


In [31]:
"""
This container class can be used to 
provide fresh iterator every time.
"""
class ReadVisits:
    def __init__(self, data_path):
        self.data_path = data_path

    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

## Item 32: Consider Generator Expressions for Large List Comprehensions

In [32]:
"""
This is a generator expression which can be used to 
avoid memory issues when reading a large file.
"""
it = ((x, x**0.5) for x in [1,2,3])

print(list(it))

[(1, 1.0), (2, 1.4142135623730951), (3, 1.7320508075688772)]


## Item 33: Compose Multiple Generators with `yield from`

In [34]:
def loop(n):
    for i in range(n):
        yield i
        
        
def multiple_loop():
    for i in loop(3):
        yield i
        
    for i in loop(2):
        yield i
        

print(list(multiple_loop()))

[0, 1, 2, 0, 1]


In [36]:
"""
'yield from' has better readability and performance.
"""
def multiple_loop_yield_from():
    yield from loop(3)
    yield from loop(2)
    

print(list(multiple_loop()))

[0, 1, 2, 0, 1]


## Item 34: Avoid Injecting Data into Generators with `send`

In [40]:
import math

def wave(amplitude, steps):
    step_size = 2 * math.pi / steps
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        output = amplitude * fraction
        yield output
    
def transmit(output):
    if output is None:
        print(f'Output is None')
    else:
        print(f'Output: {output:>5.1f}')

def run(it):
    for output in it:
        transmit(output)

run(wave(3.0, 8))

Output:   0.0
Output:   2.1
Output:   3.0
Output:   2.1
Output:   0.0
Output:  -2.1
Output:  -3.0
Output:  -2.1


### Generators and `send()` - confusing to use

In [49]:
def gen():
    received = yield 1
    print(f'received: {received}')
    
it = gen()
try:
    it.send()
except TypeError as e:
    print(f'{e!r}')

TypeError('send() takes exactly one argument (0 given)')


In [42]:
try:
    it.send('abc')
except TypeError as e:
    print(f'{e!r}')

TypeError("can't send non-None value to a just-started generator")


In [47]:
it = gen()
print(it.send(None))

try:
    it.send('abc')
except StopIteration as e:
    print(f'{e!r}')

1
received: abc
StopIteration()


### Avoid `send()` by providing an input iterator

In [59]:
def gen_with_it(it):
    received = next(it)
    yield 1
    print(f'received: {received}')
    
    
it = iter(['abc'])
gen_it = gen_with_it(it)
print(next(gen_it))
      
try:
    next(gen_it)
except StopIteration as e:
    print(f'{e!r}')

1
received: abc
StopIteration()


## Item 35: Avoid Causing State Transitions in Generators with `throw`

Just don't bother learning generators' throw: `it.throw(Error())`.

## Item 36: Consider `itertools` for Working with Iterators and Generators

Whenever you find yourself dealing with tricky iteration code, it’s worth looking at the itertools documentation again to see if there’s anything in there for you to use. 

In [61]:
import itertools 

help(itertools)

Help on built-in module itertools:

NAME
    itertools - Functional tools for creating and using iterators.

DESCRIPTION
    Infinite iterators:
    count(start=0, step=1) --> start, start+step, start+2*step, ...
    cycle(p) --> p0, p1, ... plast, p0, p1, ...
    repeat(elem [,n]) --> elem, elem, elem, ... endlessly or up to n times
    
    Iterators terminating on the shortest input sequence:
    accumulate(p[, func]) --> p0, p0+p1, p0+p1+p2
    chain(p, q, ...) --> p0, p1, ... plast, q0, q1, ... 
    chain.from_iterable([p, q, ...]) --> p0, p1, ... plast, q0, q1, ... 
    compress(data, selectors) --> (d[0] if s[0]), (d[1] if s[1]), ...
    dropwhile(pred, seq) --> seq[n], seq[n+1], starting when pred fails
    groupby(iterable[, keyfunc]) --> sub-iterators grouped by value of keyfunc(v)
    filterfalse(pred, seq) --> elements of seq where pred(elem) is False
    islice(seq, [start,] stop [, step]) --> elements from
           seq[start:stop:step]
    starmap(fun, seq) --> fun(*seq

### Linking Iterators Together

In [62]:
it = itertools.chain([1], [2, 3])
print(list(it))

[1, 2, 3]


In [63]:
it = itertools.repeat('hello', 3)
print(list(it))

['hello', 'hello', 'hello']


In [64]:
print(['hello']*3) # why use itertools?

['hello', 'hello', 'hello']


In [65]:
it = itertools.cycle([1, 2])
result = [next(it) for _ in range(10)]
print(result)

[1, 2, 1, 2, 1, 2, 1, 2, 1, 2]


In [66]:
it1, it2, it3 = itertools.tee([1, 2], 3)
print(list(it1))
print(list(it2))
print(list(it3))

[1, 2]
[1, 2]
[1, 2]


In [67]:
key = ['a', 'b', 'c']
value = [1, 2]
print(list(zip(key, value)))

[('a', 1), ('b', 2)]


In [68]:
print(list(itertools.zip_longest(key, value, fillvalue='hoho')))

[('a', 1), ('b', 2), ('c', 'hoho')]


### Filtering Items from an Iterator

In [73]:
values = range(10)

five = itertools.islice(values, 5)
print(list(five))

odds = itertools.islice(values, 1, 10, 2)
print(list(odds))

[0, 1, 2, 3, 4]
[1, 3, 5, 7, 9]


In [76]:
seven = itertools.takewhile(lambda x: x < 7, values)
print(list(seven))

[0, 1, 2, 3, 4, 5, 6]


In [77]:
from_seven = itertools.dropwhile(lambda x: x < 7, values)
print(list(from_seven))

[7, 8, 9]


In [79]:
evens = lambda x: x % 2 == 0
filter_result = filter(evens, values)
filter_false_result = itertools.filterfalse(evens, values)

print(list(filter_result))
print(list(filter_false_result))

[0, 2, 4, 6, 8]
[1, 3, 5, 7, 9]


### Producing Combinations of Items from Iterators

In [87]:
"""
itertools.accumulate() is the same as the functools.reduce()
but with outputs yielded one step at a time.
"""
values = range(10)
print(list(itertools.accumulate(values)))

[0, 1, 3, 6, 10, 15, 21, 28, 36, 45]


In [86]:
print(list(itertools.accumulate(values, lambda x, y: (x+y)%20)))

[0, 1, 3, 6, 10, 15, 1, 8, 16, 5]


In [89]:
print(list(itertools.product([1, 2], repeat=2)))

[(1, 1), (1, 2), (2, 1), (2, 2)]


In [90]:
print(list(itertools.product([1, 2], repeat=3)))

[(1, 1, 1), (1, 1, 2), (1, 2, 1), (1, 2, 2), (2, 1, 1), (2, 1, 2), (2, 2, 1), (2, 2, 2)]


In [91]:
print(list(itertools.product([1, 2], ['a', 'b'])))

[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b')]


In [92]:
print(list(itertools.permutations([1, 2, 3], 2)))

[(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)]


In [93]:
# This is the permutation with repetition.
print(list(itertools.product([1, 2, 3], repeat=2)))

[(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]


In [94]:
it = itertools.combinations([1, 2, 3], 2)
print(list(it))

[(1, 2), (1, 3), (2, 3)]


In [95]:
# This is the combination with repetition.
it = itertools.combinations_with_replacement([1, 2, 3], 2)
print(list(it))

[(1, 1), (1, 2), (1, 3), (2, 2), (2, 3), (3, 3)]
